In [ ]:
# 3.12.4
# !pip install langchain-core==0.2.38 langsmith==0.1.117 langchain-community==0.2.16 langchain-experimental==0.0.65

In [15]:
# 구글 생성형 AI 라이브러리를 가져옴
import google.generativeai as genai

#에러가 발생할 경우 자동으로 재도전하도록 함
from google.api_core import retry

# OS와 상호작용하기 위한 os 모듈을 임포트 (환경 변수나 파일 경로 관리 등에 사용)
import os

# Langchain 라이브러리에서 프롬프트 템플릿을 가져옴
from langchain import PromptTemplate

# 웹 페이지에서 문서를 로드하기 위한 WebBaseLoader를 임포트
from langchain.document_loaders import WebBaseLoader

# Langchain에서 문자열 출력을 처리하기 위한 StrOutputParser 임포트
from langchain.schema import StrOutputParser

# Langchain의 프롬프트 템플릿에서 문서 형식화를 위한 format_document 함수 임포트
from langchain.schema.prompt_template import format_document

In [16]:
# 'GOOGLE_API_KEY' 값을 가져와 API_KEY 변수에 저장
key_path = '/Users/jaesolshin/key/gemini_key.txt'
API_KEY = open(key_path, 'r', encoding='utf-8').read()

# 구글 생성형 AI 라이브러리 설정을 API_KEY로 구성
genai.configure(api_key=API_KEY)

### 웹 사이트 데이터를 읽고 요약하기

In [17]:
loader = WebBaseLoader("https://blog.google/technology/ai/google-gemini-ai/#sundar-note")
docs = loader.load()

topK: 각 토큰 선택 단계에서 확률이 가장 높은 topK 토큰이 샘플링

topP: 토큰은 확률이 가장 높은 것에서 가장 낮은 것부터 확률은 topP 값과 같습니다. 예를 들어 토큰 A, B, C의 확률은 0.3, 0.2, 0.1이고 topP 값이 0.5라면, 모델은 C를 후보에서 제외합니다. 기본 topP 값은 0.95입니다.

In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI

# 모델 매개변수를 구성하려면 'generation_config' 매개변수를 사용합니다.
# 예: generation_config = {"온도": 0.7, "topP": 0.8, "topK": 40}
# 모델에 대한 사용자 지정 온도만 설정하려는 경우
# "온도" 매개 변수를 직접 입력합니다.
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

In [20]:
# WebBaseLoader로부터 데이터 추출
doc_prompt = PromptTemplate.from_template("{page_content}")

# Gemini에 질의
llm_prompt_template = """다음에 대한 요약을 간결하게 작성합니다:

"{text}"

요약 내용:"""

# 'llm_prompt_template'로부터 프롬프트 템플릿 객체를 생성
llm_prompt = PromptTemplate.from_template(llm_prompt_template)

# 생성된 프롬프트 템플릿을 출력
print(llm_prompt)

input_variables=['text'] template='다음에 대한 요약을 간결하게 작성합니다:\n\n"{text}"\n\n요약 내용:'


In [21]:
stuff_chain = (
    # 문서로부터 데이터를 추출하고 "text"라는 키를 추가.
    {
        "text": lambda docs: "\n\n".join(
            format_document(doc, doc_prompt) for doc in docs
        )
    }
    | llm_prompt         # Prompt for Gemini
    | llm                # Gemini API function
    | StrOutputParser()  # output parser
)

In [22]:
stuff_chain.invoke(docs)

'Google은 Gemini라는 가장 강력하고 일반적인 AI 모델을 발표했습니다. Gemini는 텍스트, 코드, 오디오, 이미지, 비디오를 포함한 다양한 정보 유형을 이해하고 처리할 수 있는 다중 모드 모델입니다. Gemini는 학계의 벤치마크에서 최첨단 성능을 보였으며 특히 MMLU 및 MMMU 벤치마크에서 뛰어난 점수를 받았습니다. Gemini는 또한 코딩, 복잡한 추론 및 정보 추출에서도 뛰어난 성능을 보여줍니다.\n\nGemini는 세 가지 크기(Ultra, Pro, Nano)로 제공됩니다. Gemini Ultra는 가장 강력한 모델이며 Gemini Pro는 다양한 작업에 적합하고 Gemini Nano는 기기에서 사용하기에 가장 효율적인 모델입니다. Gemini는 현재 Google 제품(Bard, Pixel 8 Pro, Search)에서 사용 가능하며 개발자는 Google AI Studio 및 Google Cloud Vertex AI를 통해 Gemini API에 액세스할 수 있습니다.\n\nGoogle은 Gemini를 책임감 있게 개발하고 사용자의 안전과 프라이버시를 최우선으로 생각합니다. Google은 Gemini의 개발 과정에서 편향, 유해성, 사이버 공격, 설득력, 자율성을 포함한 잠재적 위험을 해결했습니다. Google은 또한 외부 전문가 및 파트너와 협력하여 Gemini의 안전성을 강화하고 있습니다.\n\nGoogle은 Gemini가 과학, 교육, 비즈니스와 같은 다양한 분야에서 혁신을 이끌어낼 잠재력이 있다고 믿습니다. Google은 Gemini를 지속적으로 개선하고 새로운 기능을 추가할 계획입니다. \n'